1. 문서의 내용을 읽는다 (docx2txt)
2. 문서를 쪼갠다 (langchain-text-splitters)
  - 토큰수 초과로 답변을 생성하지 못할수 있고
  - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장 (OpenAIEmbeddings, chroma)
4. 질문이 있을 때, 백터 테이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 (similarity_search)

In [ ]:
%pip install --upgrade --quiet  docx2txt langchain-community

In [ ]:
%pip install -qU langchain-text-splitters

In [25]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500, # 하나의 청크가 가질수있는 토큰수
  chunk_overlap=200 # 청크 간 중복 토큰수
)

loader = Docx2txtLoader("./tax.docx")

document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
len(document_list)

In [27]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# Upstage 에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model='embedding-query')

In [ ]:
%pip install langchain-chroma

In [29]:
from langchain_chroma import Chroma

# ./chroma에 저장된 내용 가져와서 필요없는건 주석

#database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="chroma_tax" persist_directory="./chroma")
database = Chroma(collection_name="chroma_tax", persist_directory="./chroma", embedding_function=embedding)

In [30]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
#retrieved_docs = database.similarity_search(query, k=3)

In [31]:
# retrieved_docs

In [32]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model="solar-pro2")


In [33]:
# prompt = f"""[Identity]
# - 당신은 최고의 소득세 전문가 입니다
# -[Context]를 참고해서 사용자의 질문에 답변해주세요

# [Context]
# {retrieved_docs}

# Question: {query}
# """

In [34]:
# ai_message = llm.invoke(prompt)

In [35]:
#ai_message.content

In [ ]:
%pip install -U langchain langchainhub --quiet

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt

In [39]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=database.as_retriever(),
  chain_type_kwargs={"prompt": prompt}
)

In [ ]:
ai_message = qa_chain({"query": query})

In [ ]:
ai_message